In [1]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 24.4 MB/s eta 0:00:0000:0100:01


In [35]:
# Zadanie 1

import pulp as pl
import numpy as np
from scipy.stats import multivariate_t


# Print generated scenarios
print("Generated Price Scenarios:")
# print(scenarios)

print("==============================================================================")

# Define the problem
model = pl.LpProblem("Maximize_Profit", pl.LpMaximize)

# Products and days
products = ['Chleb_domowy_0_5kg', 'Chleb_domowy_0_9kg', 'Chleb_na_zakwasie', 'Bulki', 'Bagietki', 'Rogaliki_z_czekolada']
days = ['Pon', 'Wt', 'Sr', 'Czw', 'Pt', 'Sob']
machines = {'Dzieza', 'Wazenie', 'Dzielarka', 'Tabula', 'Rogalikarka', 'Nadziewarka', 'Garownia', 'Piec', 'Pomieszczenie_do_studzenia', 'Pakowanie'}

# Income from product sales in PLN/piece
product_income = {'Chleb_domowy_0_5kg': 5.2, 'Chleb_domowy_0_9kg': 7.9, 'Chleb_na_zakwasie': 6.2, 'Bulki': 1.4, 'Bagietki': 1.55, 'Rogaliki_z_czekolada': 2.1}

# Time available per month (in hours)
# days_per_day = 24
shifts_per_day = 2
hours_per_shift = 8
hours_per_day = shifts_per_day * hours_per_shift

# Initial stock and storage cost
initial_stock = 0
storage_cost_per_unit_per_day = 1
storage_capacity = 3000
end_stock = 0

furnace_cost_per_hour = 82.4 # https://giko.pl/kalkulator-zuzycia-energii-dla-piecow-piekarniczych/

# Production times required per product in hours
production_times = {
    'Chleb_domowy_0_5kg': {'Dzieza': 4.7, 'Wazenie': 0.29, 'Dzielarka': 0.0, 'Tabula': 0.7, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.0, 'Piec': 0.68, 'Pomieszczenie_do_studzenia': 1.8, 'Pakowanie': 0.56},
    'Chleb_na_zakwasie': {'Dzieza': 8.6, 'Wazenie': 0.22, 'Dzielarka': 0.0, 'Tabula': 0.6, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.4, 'Piec': 1.0, 'Pomieszczenie_do_studzenia': 2.3, 'Pakowanie': 0.6},
    'Chleb_domowy_0_9kg': {'Dzieza': 5, 'Wazenie': 0.2, 'Dzielarka': 0.0, 'Tabula': 0.55, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 1.5, 'Piec': 1.2, 'Pomieszczenie_do_studzenia': 2.5, 'Pakowanie': 0.57},
    'Bulki': {'Dzieza': 1, 'Wazenie': 0.0, 'Dzielarka': 0.23, 'Tabula': 0.34, 'Rogalikarka': 0.0, 'Nadziewarka': 0.0, 'Garownia': 0.38, 'Piec': 0.25, 'Pomieszczenie_do_studzenia': 0.48, 'Pakowanie': 0.0},
    'Bagietki': {'Dzieza': 0.95, 'Wazenie': 0.0, 'Dzielarka': 0.24, 'Tabula': 0.0, 'Rogalikarka': 0.28, 'Nadziewarka': 0.0, 'Garownia': 0.33, 'Piec': 0.23, 'Pomieszczenie_do_studzenia': 0.42, 'Pakowanie': 0.0},
    'Rogaliki_z_czekolada': {'Dzieza': 0.55, 'Wazenie': 0.0, 'Dzielarka': 0.28, 'Tabula': 0.0, 'Rogalikarka': 0.31, 'Nadziewarka': 0.4, 'Garownia': 0.25, 'Piec': 0.18, 'Pomieszczenie_do_studzenia': 0.82, 'Pakowanie': 0.42}
}

# Market limits for sales of Products
market_limits = {
    "Chleb_domowy_0_5kg": {"Pon": 200, "Wt": 30, "Sr": 0, "Czw": 20, "Pt": 30, "Sob": 18},
    "Chleb_domowy_0_9kg": {"Pon": 150, "Wt": 25, "Sr": 80, "Czw": 150, "Pt": 250, "Sob": 180},
    "Chleb_na_zakwasie": {"Pon": 180, "Wt": 280, "Sr": 0, "Czw": 180, "Pt": 280, "Sob": 18},
    "Bulki": {"Pon": 400, "Wt": 500, "Sr": 0, "Czw": 40, "Pt": 50, "Sob": 180},
    "Bagietki": {"Pon": 350, "Wt": 450, "Sr": 0, "Czw": 350, "Pt": 45, "Sob": 180},
    "Rogaliki_z_czekolada": {"Pon": 300, "Wt": 400, "Sr": 0, "Czw": 300, "Pt": 400, "Sob": 180},
}

batch_size = {
    "Chleb_domowy_0_5kg": 240,
    "Chleb_domowy_0_9kg": 130,
    "Chleb_na_zakwasie": 200,
    "Bulki": 200,
    "Bagietki": 200,
    "Rogaliki_z_czekolada": 200,
}


# Machines available
machines_available = {"Dzieza": 10, "Wazenie": 3, "Dzielarka": 3, "Tabula": 20, "Rogalikarka": 1, "Nadziewarka": 1, "Garownia": 3, "Piec": 5, "Pomieszczenie_do_studzenia": 4, "Pakowanie": 2}

# Variables for production, sales, and storage of products each day
production_vars = pl.LpVariable.dicts("Production", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
batch_vars = pl.LpVariable.dicts("Batches", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
sales_vars = pl.LpVariable.dicts("Sales", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
storage_vars = pl.LpVariable.dicts("Storage", ((product, day) for product in products for day in days), lowBound=0, upBound=storage_capacity, cat='Integer')

# Constrain for P4 sales based on P1 and P2 sales
# for day in days:
#   model += sales_vars['P4', day] >= sales_vars['P1', day] + sales_vars['P2', day]

# Inventory and sales constraints
for product in products:
    model += storage_vars[product, 'Pon'] == initial_stock + production_vars[product, 'Pon'] * batch_size[product] - sales_vars[product, 'Pon']  # for Pon
    for i in range(1, len(days)):
        day = days[i]
        prev_day = days[i-1]
        model += storage_vars[product, day] == storage_vars[product, prev_day] + production_vars[product, day] * batch_size[product] - sales_vars[product, day]  # for rest of the days
    # model += storage_vars[product, 'Sob'] == end_stock  # Ending stock

# Machine time constraints for each product and day
for day in days:
    total_production_time = sum(production_vars[product, day] * production_times[product][machine] / machines_available[machine]
                                for product in products
                                for machine in machines)
    model += total_production_time <= hours_per_day, f"Global_production_time_{day}"
    model += storage_capacity >= sum(storage_vars[product, day] for product in products for day in days)

# Market sales constraints
for product in products:
    for day in days:
        # model += sales_vars[product, day] <= market_limits[product][day], f"Max_sales_{product}_{day}"
        model += sales_vars[product, day] == market_limits[product][day], f"Max_sales_{product}_{day}"
        model += sales_vars[product, day] <= production_vars[product, day] * batch_size[product] + (storage_vars[product, days[days.index(day) - 1]] if day != 'Pon' else initial_stock)
        model += batch_vars[product, day] * batch_size[product] >= production_vars[product, day]

# model += batch_vars *


# Objective function: Maximize profit from sales minus storage costs
model += pl.lpSum([sales_vars[product, day] * product_income[product] \
                   - storage_vars[product, day] * storage_cost_per_unit_per_day for product in products for day in days]) \
                   - pl.lpSum([batch_vars[product, day] * production_times[product]['Piec'] * furnace_cost_per_hour for product in products for day in days]), "Total_Profit"

# Solve the model
model.solve()

# Check the status of the solution and print the results
print("Status:", pl.LpStatus[model.status])
if pl.LpStatus[model.status] == 'Optimal':
    print("Total Profit:", pl.value(model.objective))

    # Printing headers
    print(f"{'Product':<10} {'day':<10} {'Production':<12} {'Sales':<10} {'Storage':<10}")

    # Display the results in tabular form
    for product in products:
        for day in days:
            production = production_vars[product, day].varValue
            sales = sales_vars[product, day].varValue
            storage = storage_vars[product, day].varValue
            print(f"{product:<10} {day:<10} {production:<12} {sales:<10} {storage:<10}")

    print("\nProduction Time Usage per Product and Day:")
    for day in days:
        print(f"\Day: {day}")
        for product in products:
            total_production_time = sum(
                production_vars[product, day].varValue * production_times[product].get(machine, 0)
                for machine in machines #.keys()
            )
            print(f"  Product: {product}, Total Production Time: {total_production_time} hours")

else:
    print("No optimal solution was found.")

Generated Price Scenarios:
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gromek/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/9a176e7c176c4f86b06d46c731e58964-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/9a176e7c176c4f86b06d46c731e58964-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 161 COLUMNS
At line 1158 RHS
At line 1315 BOUNDS
At line 1460 ENDATA
Problem MODEL has 156 rows, 144 columns and 600 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 21040 - 0.00 seconds
Cgl0003I 0 fixed, 116 tightened bounds, 39 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 5 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 85 rows, 91 columns (91 integer (0 of which binary)) and 254 elements
Cbc0031I 24 added rows had average density of 6.7083333
Cbc0